# Efficient summarization
* using map-reduce https://python.langchain.com/docs/use_cases/summarization#option-2.-map-reduce https://python.langchain.com/docs/modules/chains/document/map_reduce
* simply split by X chars

In [ ]:
# import locale
# locale.getpreferredencoding = lambda x: "UTF-8"

In [ ]:
!pip install -q --upgrade transformers langchain tiktoken accelerate arize-phoenix[experimental] auto-gptq optimum  # for GPTQ

In [4]:
!poetry add langchain
!poetry add lxml  # for html splitter
!poetry add transformers
!poetry add arize-phoenix\[experimental\]
!poetry add sentence-transformers
# tqdm \  # for langchain progress

The following packages are already present in the pyproject.toml and will be skipped:

  • langchain

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
The following packages are already present in the pyproject.toml and will be skipped:

  • lxml

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
The following packages are already present in the pyproject.toml and will be skipped:

  • transformers

If you want to update it to the latest compatible version, you can use `poetry update package`.
If you prefer to upgrade it to the latest available version, you can use `poetry add package@latest`.

Nothing to add.
Using version ^2.1.0 for arize-phoenix

Updating de

In [1]:
import phoenix as px
session = px.launch_app()
px.active_session().view()

from phoenix.trace.langchain import OpenInferenceTracer, LangChainInstrumentor
tracer = OpenInferenceTracer()
LangChainInstrumentor(tracer).instrument()

# https://zenn.dev/ohtaman/articles/run_webapp_on_colab
# from google.colab import output
# output.serve_kernel_port_as_iframe(6006, path='/')

/Users/hide/llm-notebooks/.venv/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🌍 To view the Phoenix app in your browser, visit http://127.0.0.1:6006/
📺 To view the Phoenix app in a notebook, run `px.active_session().view()`
📖 For more information on how to use Phoenix, check out https://docs.arize.com/phoenix
📺 Opening a view to the Phoenix app. The app is running at http://127.0.0.1:6006/


In [2]:
%run loader.ipynb

In [3]:
html_header_splits = load_and_split(**TEST_URLS['goldman'])
[h.metadata for h in html_header_splits]

Fetching pages: 100%|##########################################################################################| 1/1 [00:00<00:00,  3.51it/s]


132028


[{},
 {'Header 2': 'Executive summary'},
 {'Header 2': 'Introduction'},
 {'Header 2': 'AI and geopolitics'},
 {'Header 2': 'The AI incumbents: The US & China'},
 {'Header 2': 'The AI capabilities of geopolitical swing states'},
 {'Header 2': 'The European Union'},
 {'Header 2': 'The United Kingdom'},
 {'Header 2': 'The United Arab Emirates'},
 {'Header 2': 'Israel'},
 {'Header 2': 'Japan, The Netherlands, South Korea, and Taiwan'},
 {'Header 2': 'India'},
 {'Header 2': 'The emerging AI powers'},
 {'Header 2': 'The geopolitics of AI governance'},
 {'Header 2': 'The emerging AI powers'},
 {'Header 2': 'How technological development drives geopolitics'},
 {'Header 2': 'The key debate: scale up or scale down?'},
 {'Header 2': 'The future of AI governance'},
 {'Header 2': 'Charting the chip wars'},
 {'Header 2': 'AI and commerce'},
 {'Header 2': 'Energy'},
 {'Header 2': 'Compute'},
 {'Header 2': 'Data'},
 {'Header 2': 'Models'},
 {'Header 2': 'What to watch'}]

## Create local LLM class
https://www.mattari-benkyo-note.com/2023/05/19/langchain_hugging_face/

In [6]:
from langchain.callbacks.manager import CallbackManager
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler
from langchain.llms import Ollama

llm = Ollama(model="llama2:7b-chat")

In [18]:
# test
# test_input = "hello, how are you?"
# print(llm(test_input))

Hello! I'm just an AI, I don't have feelings or emotions like humans do, so I can't really experience the world in the same way that you do. However, I'm here to help you with any questions or tasks you might have, so feel free to ask me anything! Is there something specific you'd like to know or talk about?


In [7]:
from langchain.chains import MapReduceDocumentsChain, ReduceDocumentsChain
from langchain.chains.combine_documents.stuff import StuffDocumentsChain
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate

# Prompt for llama2
# * https://huggingface.co/TheBloke/Llama-2-13B-chat-GPTQ/discussions/5
# * https://huggingface.co/blog/llama2#how-to-prompt-llama-2
# * https://discuss.huggingface.co/t/trying-to-understand-system-prompts-with-llama-2-and-transformers-interface/59016/8

# Map
map_template = """<s>[INST]The following is a set of documents:

{docs}

Based on this list of docs, please identify the main themes.[/INST]
Helpful Answer:"""

map_prompt = PromptTemplate.from_template(map_template)
map_chain = LLMChain(llm=llm, prompt=map_prompt)

# Reduce
reduce_template = """<s>[INST]The following is set of summaries:

{docs}

Take these and distill it into a final, consolidated summary of the main themes.[/INST]
Helpful Answer:"""
reduce_prompt = PromptTemplate.from_template(reduce_template)
reduce_chain = LLMChain(llm=llm, prompt=reduce_prompt)

## Takes a list of documents, combines them into a single string, and passes this to an LLMChain
combine_documents_chain = StuffDocumentsChain(
    llm_chain=reduce_chain, document_variable_name="docs"
)

# Reduce Doc chain
## Combines and iteravely reduces the mapped documents
reduce_documents_chain = ReduceDocumentsChain(
    combine_documents_chain=combine_documents_chain,
    collapse_documents_chain=combine_documents_chain,
    token_max=2000,
)

# Map-Reduce chain
## Combining documents by mapping a chain over them, then combining results
map_reduce_chain = MapReduceDocumentsChain(
    llm_chain=map_chain,
    reduce_documents_chain=reduce_documents_chain,
    document_variable_name="docs",
    return_intermediate_steps=False,
)

In [16]:
# 1300 secs... usng Ollama
print(map_reduce_chain.run(html_header_splits))

Token indices sequence length is longer than the specified maximum sequence length for this model (6246 > 1024). Running this sequence through the model will result in indexing errors


Based on the provided documents, the main themes that emerge in the geopolitics of AI governance are:

1. The impact of AI on global power dynamics and the growing role of countries like India and Israel in the global AI ecosystem.
2. The need for international cooperation to address the risks associated with AI, such as misinformation, autonomous weapons systems, and exploitation by non-state actors.
3. The importance of developing norms and standards for AI governance, led by likeminded countries such as the G7-led Hiroshima Process.
4. The potential for AI to be used for malicious purposes, such as creating harmful weapons or amplifying misinformation.
5. The unpredictability and bias of AI algorithms, leading to unintended consequences and discriminatory outcomes in areas like law enforcement, healthcare, and job recruitment.
6. The possibility of an "intelligence explosion" where AI could operate with objectives misaligned with universal values of human dignity.
7. The need for re

In [24]:
# px.active_session().get_spans_dataframe().to_feather('20231224-goldman-jared-report.feather')